In [1]:
import tensorflow as tf
import numpy as np
import random

### Logic gate definition

In [2]:
gate_and  = lambda x: x[0] and x[1]
gate_or   = lambda x: x[0] or  x[1]
gate_nand = lambda x: 1 - gate_and(x)
gate_nor  = lambda x: 1 - gate_or(x)
gate_xor  = lambda x: gate_and((gate_or(x), gate_nand(x)))

### Data generation

In [3]:
gate_list = [('AND', gate_and), ('OR', gate_or), ('NAND', gate_nand), ('NOR', gate_nor), ('XOR', gate_xor)]
x_list = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]]).astype(np.float32)
y_list = {}

for gate_name, gate_body in gate_list:        
    y_list[gate_name] = np.array(map(gate_body, [(x[0], x[1]) for x in x_list])).reshape(4, 1)

### Multi-task neural network design

In [4]:
weights = {
    'h1_W':     tf.Variable(tf.random_uniform([2, 5], -1.0, 1.0)),
    'h2_W':     tf.Variable(tf.random_uniform([5, 4], -1.0, 1.0)),
    'y_and_W':  tf.Variable(tf.random_uniform([4, 1], -1.0, 1.0)),
    'y_or_W':   tf.Variable(tf.random_uniform([4, 1], -1.0, 1.0)),
    'y_nand_W': tf.Variable(tf.random_uniform([4, 1], -1.0, 1.0)),
    'y_nor_W':  tf.Variable(tf.random_uniform([4, 1], -1.0, 1.0)),
    'y_xor_W':  tf.Variable(tf.random_uniform([4, 1], -1.0, 1.0))
}

biases = {
    'h1_b':     tf.Variable(tf.zeros([5])),
    'h2_b':     tf.Variable(tf.zeros([4])),
    'y_and_b':  tf.Variable(tf.zeros([1])),
    'y_or_b':   tf.Variable(tf.zeros([1])),
    'y_nand_b': tf.Variable(tf.zeros([1])),
    'y_nor_b':  tf.Variable(tf.zeros([1])),
    'y_xor_b':  tf.Variable(tf.zeros([1]))
}

In [5]:
x = tf.placeholder(tf.float32)

h1 = tf.sigmoid(tf.matmul(x, weights['h1_W']) + biases['h1_b'])

h2 = tf.sigmoid(tf.matmul(h1, weights['h2_W']) + biases['h2_b'])

y_and_pred  = tf.sigmoid(tf.matmul(h2, weights['y_and_W'])  + biases['y_and_b'])
y_or_pred   = tf.sigmoid(tf.matmul(h2, weights['y_or_W'])   + biases['y_or_b'])
y_nand_pred = tf.sigmoid(tf.matmul(h2, weights['y_nand_W']) + biases['y_nand_b'])
y_nor_pred  = tf.sigmoid(tf.matmul(h2, weights['y_nor_W'])  + biases['y_nor_b'])
y_xor_pred  = tf.sigmoid(tf.matmul(h2, weights['y_xor_W'])  + biases['y_xor_b'])

y_and_true  = tf.placeholder(tf.float32)
y_or_true   = tf.placeholder(tf.float32)
y_nand_true = tf.placeholder(tf.float32)
y_nor_true  = tf.placeholder(tf.float32)
y_xor_true  = tf.placeholder(tf.float32)

loss_and   = tf.reduce_mean(-y_and_true  * tf.log(y_and_pred)  - (1-y_and_true)  * tf.log(1-y_and_pred))
loss_or    = tf.reduce_mean(-y_or_true   * tf.log(y_or_pred)   - (1-y_or_true)   * tf.log(1-y_or_pred))
loss_nand  = tf.reduce_mean(-y_nand_true * tf.log(y_nand_pred) - (1-y_nand_true) * tf.log(1-y_nand_pred))
loss_nor   = tf.reduce_mean(-y_nor_true  * tf.log(y_nor_pred)  - (1-y_nor_true)  * tf.log(1-y_nor_pred))
loss_xor   = tf.reduce_mean(-y_xor_true  * tf.log(y_xor_pred)  - (1-y_xor_true)  * tf.log(1-y_xor_pred))
loss_total = loss_and + loss_or + loss_nand + loss_nor + loss_xor

correct_xor  = tf.equal(tf.equal(y_xor_true, 1.0), tf.greater(y_xor_pred, 0.5))
accuracy_xor = tf.reduce_mean(tf.cast(correct_xor, tf.float32))

### Data generation

In [6]:
train_data = {
    x:           np.concatenate([x_list         for _ in range(10)]),
    y_and_true:  np.concatenate([y_list['AND']  for _ in range(10)]),
    y_or_true:   np.concatenate([y_list['OR']   for _ in range(10)]),
    y_nand_true: np.concatenate([y_list['NAND'] for _ in range(10)]),
    y_nor_true:  np.concatenate([y_list['NOR']  for _ in range(10)]),
    y_xor_true:  np.concatenate([y_list['XOR']  for _ in range(10)])}

In [7]:
test_data = {
    x:x_list,
    y_and_true: y_list['AND'],
    y_or_true:  y_list['OR'],
    y_nand_true:y_list['NAND'],
    y_nor_true: y_list['NOR'],
    y_xor_true: y_list['XOR']}

### Experiment

In [8]:
def print_loss(sess, iter):
    print('%d %f %f %f %f %f'%(
        iter,
        sess.run(loss_and, train_data),
        sess.run(loss_or, train_data),
        sess.run(loss_nand, train_data),
        sess.run(loss_nor, train_data),
        sess.run(loss_xor, train_data)))
    
def dnn_experiment(train_op_list):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        num_iter = 1000
        for i in range(num_iter):
            sess.run(random.sample(train_op_list, 1), train_data)
            if accuracy_xor.eval(test_data) == 1.0:
                print_loss(sess, i+1)
                return

In [9]:
train_op_and   = tf.train.AdamOptimizer(tf.Variable(0.1)).minimize(loss_and)
train_op_or    = tf.train.AdamOptimizer(tf.Variable(0.1)).minimize(loss_or)
train_op_nand  = tf.train.AdamOptimizer(tf.Variable(0.1)).minimize(loss_nand)
train_op_nor   = tf.train.AdamOptimizer(tf.Variable(0.1)).minimize(loss_nor)
train_op_xor   = tf.train.AdamOptimizer(tf.Variable(0.1)).minimize(loss_xor)
train_op_total = tf.train.AdamOptimizer(tf.Variable(0.1)).minimize(loss_total)

### XOR only

In [10]:
dnn_experiment([train_op_xor])

39 0.773797 1.001325 0.738363 0.597698 0.386693


### Alternate MTL

In [11]:
dnn_experiment([train_op_and, train_op_or, train_op_nand, train_op_nor, train_op_xor])

295 0.082623 0.090572 0.072079 0.044362 0.414339


### Joint MTL

In [12]:
dnn_experiment([train_op_total])

53 0.039241 0.033578 0.053141 0.047369 0.455856
